Class Definitions

In [30]:
class LinkedListNode:
    """Node for linked list to store edges."""
    def __init__(self, value):
        self.value = value
        self.next = None


class Graph:
    """Graph implemented with adjacency list using linked lists."""
    def __init__(self, num_nodes):
        self.num_nodes = num_nodes
        self.adj_list = [None for _ in range(num_nodes)]

    def add_edge(self, from_node, to_node):
        """Add a directed edge from 'from_node' to 'to_node'."""
        new_node = LinkedListNode(to_node)
        new_node.next = self.adj_list[from_node]
        self.adj_list[from_node] = new_node

    def get_neighbors(self, node):
        """Get outgoing neighbors of a node."""
        neighbors = []
        current = self.adj_list[node]
        while current:
            neighbors.append(current.value)
            current = current.next
        return neighbors

    def display_graph(self):
        """Display the adjacency list of the graph."""
        for i in range(self.num_nodes):
            print(f"Node {i}:", end=" ")
            current = self.adj_list[i]
            while current:
                print(f"{current.value}", end=" -> ")
                current = current.next
            print("None")


class QueueNode:
    """Node for the queue."""
    def __init__(self, value):
        self.value = value
        self.next = None


class Queue:
    """Queue implemented with linked list."""
    def __init__(self):
        self.front = None
        self.rear = None

    def enqueue(self, value):
        """Add a value to the rear of the queue."""
        new_node = QueueNode(value)
        if self.rear:
            self.rear.next = new_node
        self.rear = new_node
        if not self.front:
            self.front = new_node

    def dequeue(self):
        """Remove and return the front value from the queue."""
        if not self.front:
            return None
        value = self.front.value
        self.front = self.front.next
        if not self.front:
            self.rear = None
        return value

    def is_empty(self):
        """Check if the queue is empty."""
        return self.front is None


class StackNode:
    """Node for the stack."""
    def __init__(self, value):
        self.value = value
        self.next = None


class Stack:
    """Stack implemented with linked list."""
    def __init__(self):
        self.top = None

    def push(self, value):
        """Add a value to the top of the stack."""
        new_node = StackNode(value)
        new_node.next = self.top
        self.top = new_node

    def pop(self):
        """Remove and return the top value from the stack."""
        if not self.top:
            return None
        value = self.top.value
        self.top = self.top.next
        return value

    def is_empty(self):
        """Check if the stack is empty."""
        return self.top is None

class HashTable:
    """Custom Hash Table implementation to store node ranks."""
    def __init__(self, size=100):
        self.size = size
        self.table = [[] for _ in range(size)]

    def _hash(self, key):
        """Generate a hash for a given key."""
        return hash(key) % self.size

    def set(self, key, value):
        """Set a key-value pair in the hash table."""
        hashed_key = self._hash(key)
        for i, (k, _) in enumerate(self.table[hashed_key]):
            if k == key:
                self.table[hashed_key][i] = (key, value)
                return
        self.table[hashed_key].append((key, value))

    def get(self, key):
        """Get the value associated with a key in the hash table."""
        hashed_key = self._hash(key)
        for k, v in self.table[hashed_key]:
            if k == key:
                return v
        raise KeyError(f"Key {key} not found in HashTable.")

    def keys(self):
        """Get all the keys in the hash table."""
        return [k for bucket in self.table for k, _ in bucket]

    def items(self):
        """Get all key-value pairs in the hash table."""
        return [(k, v) for bucket in self.table for k, v in bucket]

    def __contains__(self, key):
        """Check if the key exists in the hash table."""
        hashed_key = self._hash(key)
        return any(k == key for k, _ in self.table[hashed_key])


Function Definitions

In [31]:
def bfs_pagerank(graph, damping=0.85, max_iter=100, tol=1e-6):
    num_nodes = graph.num_nodes
    ranks = HashTable(size=num_nodes)
    
    # Initialize ranks for each node to 1 / num_nodes
    node = 0
    while node < num_nodes:
        ranks.set(node, 1 / num_nodes)
        node += 1

    teleport_prob = (1 - damping) / num_nodes
    iteration = 0

    while iteration < max_iter:
        new_ranks = HashTable(size=num_nodes)
        node = 0
        while node < num_nodes:
            new_ranks.set(node, teleport_prob)
            node += 1

        # Queue for BFS traversal
        queue = Queue()
        node = 0
        while node < num_nodes:
            queue.enqueue(node)
            node += 1

        visited = set()

        while not queue.is_empty():
            current_node = queue.dequeue()
            if current_node not in visited:
                visited.add(current_node)
                neighbors = graph.get_neighbors(current_node)
                neighbor_count = len(neighbors)
                
                if neighbor_count > 0:
                    share = ranks.get(current_node) / neighbor_count
                    for neighbor in neighbors:
                        new_ranks.set(neighbor, new_ranks.get(neighbor) + damping * share)
                        if neighbor not in visited:
                            queue.enqueue(neighbor)

        # Check for convergence
        node = 0
        diff = 0
        while node < num_nodes:
            diff += abs(new_ranks.get(node) - ranks.get(node))
            node += 1

        if diff < tol:
            print(f"BFS PageRank converged after {iteration + 1} iterations")
            break

        ranks = new_ranks
        iteration += 1

    return ranks


    return ranks

def dfs_pagerank(graph, damping=0.85, max_iter=100, tol=1e-6):
    num_nodes = graph.num_nodes
    ranks = HashTable(size=num_nodes)

    # Initialize ranks for each node to 1 / num_nodes
    node = 0
    while node < num_nodes:
        ranks.set(node, 1 / num_nodes)
        node += 1

    teleport_prob = (1 - damping) / num_nodes
    iteration = 0

    while iteration < max_iter:
        new_ranks = HashTable(size=num_nodes)
        node = 0
        while node < num_nodes:
            new_ranks.set(node, teleport_prob)
            node += 1

        # Stack for DFS traversal
        stack = Stack()
        node = 0
        while node < num_nodes:
            stack.push(node)
            node += 1

        visited = set()

        while not stack.is_empty():
            current_node = stack.pop()
            if current_node not in visited:
                visited.add(current_node)
                neighbors = graph.get_neighbors(current_node)
                neighbor_count = len(neighbors)

                if neighbor_count > 0:
                    share = ranks.get(current_node) / neighbor_count
                    for neighbor in neighbors:
                        new_ranks.set(neighbor, new_ranks.get(neighbor) + damping * share)
                        if neighbor not in visited:
                            stack.push(neighbor)

        # Check for convergence
        node = 0
        diff = 0
        while node < num_nodes:
            diff += abs(new_ranks.get(node) - ranks.get(node))
            node += 1

        if diff < tol:
            print(f"DFS PageRank converged after {iteration + 1} iterations")
            break

        ranks = new_ranks
        iteration += 1

    return ranks

def display_pagerank(ranks, method):
    """Display PageRank scores."""
    print(f"\n{method} PageRank Scores:")
    for node, rank in sorted(ranks.items(), key=lambda item: item[1], reverse=True):
        print(f"Node {node}: Rank = {rank:.6f}")



Graph Creation

In [32]:
# Create a graph with 10 nodes
num_nodes = 10
graph = Graph(num_nodes)

# Add edges to the graph to increase complexity
graph.add_edge(0, 1)
graph.add_edge(0, 3)
graph.add_edge(1, 2)
graph.add_edge(1, 4)
graph.add_edge(2, 5)
graph.add_edge(3, 4)
graph.add_edge(3, 6)
graph.add_edge(4, 5)
graph.add_edge(4, 7)
graph.add_edge(5, 8)
graph.add_edge(6, 7)
graph.add_edge(7, 8)
graph.add_edge(7, 9)
graph.add_edge(8, 9)
graph.add_edge(9, 6)

# Display the updated graph
print("Initial Graph:")
graph.display_graph()



Initial Graph:
Node 0: 3 -> 1 -> None
Node 1: 4 -> 2 -> None
Node 2: 5 -> None
Node 3: 6 -> 4 -> None
Node 4: 7 -> 5 -> None
Node 5: 8 -> None
Node 6: 7 -> None
Node 7: 9 -> 8 -> None
Node 8: 9 -> None
Node 9: 6 -> None


Function Execution

In [33]:
# Computes and display BFS-based PageRank
bfs_ranks = bfs_pagerank(graph)
display_pagerank(bfs_ranks, "BFS-Based")
print("")
# Computes and displays DFS-based PageRank
dfs_ranks = dfs_pagerank(graph)
display_pagerank(dfs_ranks, "DFS-Based")


BFS PageRank converged after 43 iterations

BFS-Based PageRank Scores:
Node 9: Rank = 0.237389
Node 6: Rank = 0.225865
Node 7: Rank = 0.221082
Node 8: Rank = 0.151093
Node 5: Rank = 0.049568
Node 4: Rank = 0.033169
Node 2: Rank = 0.024084
Node 1: Rank = 0.021375
Node 3: Rank = 0.021375
Node 0: Rank = 0.015000

DFS PageRank converged after 43 iterations

DFS-Based PageRank Scores:
Node 9: Rank = 0.237389
Node 6: Rank = 0.225865
Node 7: Rank = 0.221082
Node 8: Rank = 0.151093
Node 5: Rank = 0.049568
Node 4: Rank = 0.033169
Node 2: Rank = 0.024084
Node 1: Rank = 0.021375
Node 3: Rank = 0.021375
Node 0: Rank = 0.015000
